In [28]:
%pip install langchain lancedb langchain_community requests langchainhub
%pip install langchain google-generativeai
%pip install faiss-cpu lancedb tiktoken
%pip install python-docx

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/lancedb/__init__.py:220: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [29]:
import requests
# Step 1: Fetch data from the Google Sheets Web App URL, which is a public API, easy right
URL = "https://script.google.com/macros/s/AKfycbwhTV8Ryxe_5wF-3fBsI7R8VVVQ_dPebAVOcnvLplvAtJZceD2KCs8D5Yj8VfcDUWWYDQ/exec"
response = requests.get(URL)

# Check if the request was successful
#in later stages, sometime if this check is not done, it will throw an error, so its better to check now
if response.status_code == 200:
    sheet_data = response.json()  # List of dictionaries (data from Google Sheets)
    print("Fetched data from Google Sheets:")
    print(sheet_data)
else:
    print("Failed to fetch data. Status code:", response.status_code)


Fetched data from Google Sheets:
[{'Timestamp': '2025-05-07T00:00:00.000Z', 'Name': 'Riya Reddy', 'Feedback': 'The syllabus is well structured and up to date.'}, {'Timestamp': '2025-05-07T00:01:00.000Z', 'Name': 'Kabir Sharma', 'Feedback': 'Some topics feel outdated and irrelevant.'}, {'Timestamp': '2025-05-07T00:02:00.000Z', 'Name': 'Maya Das', 'Feedback': 'The syllabus is well structured and up to date.'}, {'Timestamp': '2025-05-07T00:03:00.000Z', 'Name': 'Anjali Singh', 'Feedback': 'The grading seems inconsistent at times.'}, {'Timestamp': '2025-05-07T00:04:00.000Z', 'Name': 'Rohit Patel', 'Feedback': 'Lab instruments are outdated.'}, {'Timestamp': '2025-05-07T00:05:00.000Z', 'Name': 'Kunal Sharma', 'Feedback': 'Quick response from the faculty to doubts.'}, {'Timestamp': '2025-05-07T00:06:00.000Z', 'Name': 'Kabir Reddy', 'Feedback': 'Classroom is too noisy and affects concentration.'}, {'Timestamp': '2025-05-07T00:07:00.000Z', 'Name': 'Kabir Singh', 'Feedback': 'Group discussions ar

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
reviews = [entry['Feedback'] for entry in sheet_data]
reviews_text = "\n".join([f"Timestamp: {entry['Timestamp']}, Name: {entry['Name']}, Feedback: {entry['Feedback']}" for entry in sheet_data])

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(reviews_text)

In [32]:
print(f"Split into {len(chunks)} chunks:")
for chunk in chunks[:5]:  # Just printing the first 5 chunks for now
    print(chunk)

Split into 51 chunks:
Timestamp: 2025-05-07T00:00:00.000Z, Name: Riya Reddy, Feedback: The syllabus is well structured and up to date.
Timestamp: 2025-05-07T00:01:00.000Z, Name: Kabir Sharma, Feedback: Some topics feel outdated and irrelevant.
Timestamp: 2025-05-07T00:02:00.000Z, Name: Maya Das, Feedback: The syllabus is well structured and up to date.
Timestamp: 2025-05-07T00:03:00.000Z, Name: Anjali Singh, Feedback: The grading seems inconsistent at times.
Timestamp: 2025-05-07T00:04:00.000Z, Name: Rohit Patel, Feedback: Lab instruments are outdated.
Timestamp: 2025-05-07T00:05:00.000Z, Name: Kunal Sharma, Feedback: Quick response from the faculty to doubts.
Timestamp: 2025-05-07T00:06:00.000Z, Name: Kabir Reddy, Feedback: Classroom is too noisy and affects concentration.
Timestamp: 2025-05-07T00:07:00.000Z, Name: Kabir Singh, Feedback: Group discussions are very effective.
Timestamp: 2025-05-07T00:08:00.000Z, Name: Tara Reddy, Feedback: Great examples, but sometimes the pace is too 

In [33]:
#embed
from langchain.embeddings import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [34]:
#embedding and store in LanceDB
from langchain.vectorstores import LanceDB
vectorDB=LanceDB.from_texts(chunks,embedding)

In [ ]:
#setting up Google Gemini as LLM
import os
from langchain_google_genai import GoogleGenerativeAI 
os.environ["GOOGLE_API_KEY"] = "enter_your_api_key"
llm=GoogleGenerativeAI(model="gemini-2.0-flash-lite",)

In [39]:
#QnA pipepline using retriever
from langchain.chains import RetrievalQA
QnA=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorDB.as_retriever(),
)

In [40]:
#query(USP of the product)
#1st try:Query for LLM retrieving our text.
query="Cluster all the reviews into 5 major categories and give a summary of each category"
result=QnA.run(query)
print(result)

Here's a breakdown of the feedback into 5 major categories with summaries:

1.  **Content Relevance/Quality:**
    *   **Summary:** Concerns about the relevance of topics and outdated material.
    *   **Examples:** "Some topics feel outdated and irrelevant."

2.  **Teaching Method/Pace:**
    *   **Summary:** Feedback related to the pace of lectures, difficulty following, and the need for more practical sessions.
    *   **Examples:** "Great examples, but sometimes the pace is too fast.", "Sometimes the lectures feel rushed and hard to follow.", "Too much theory, needs more practical sessions."

3.  **Support/Accessibility:**
    *   **Summary:** Issues with getting help outside of class hours.
    *   **Examples:** "Hard to get any help outside of class hours."

4.  **Engagement/Classroom Environment:**
    *   **Summary:** Positive and negative feedback regarding the classroom environment and activities, specifically group discussions and noise levels.
    *   **Examples:** "Group d

Retrivel QnA may sound good, but at the end its a Question-Answer pipeline. To do the holistic clustering try to do cosine similarity+ KMeans clustering

In [41]:
#2nd try:cosine similarity+KMeans clustering

from sklearn.cluster import KMeans

embedded_vectors =embedding.embed_documents(chunks)
# 5 groups, we can even change as per need by developer
kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(embedded_vectors)

# Group original chunks by cluster label
clustered_reviews = {i: [] for i in range(5)}
for label, chunk in zip(labels, chunks):
    clustered_reviews[label].append(chunk)


In [42]:
cluster_summaries = {}
for i, texts in clustered_reviews.items():
    combined_text = "\n".join(texts)
    prompt = f"Summarize the following feedback reviews:\n{combined_text}"
    response = llm.invoke(prompt)
    cluster_summaries[i] = response
    print(f"\n Cluster {i} Summary:\n{response}\n")


 Cluster 0 Summary:
Here's a summary of the feedback, highlighting the main themes:

**Positive Feedback:**

*   **Clear Explanations:** Several reviewers (Maya Reddy, Rohit Das, Sneha Das) praised the teacher's ability to explain concepts clearly.
*   **Fair Exams:**  Two reviewers (Anjali Patel, Riya Das) found the exams fair and reflective of the coursework.
*   **Real-World Case Studies:** Two reviewers (Sneha Das, Rohit Reddy) appreciated the inclusion of real-world case studies.
*   **Well Structured Syllabus:** One reviewer (Riya Das) found the syllabus well structured and up to date.

**Negative Feedback/Areas for Improvement:**

*   **Pace Too Fast:** The most frequent complaint (Tara Reddy, Sneha Sharma, Kabir Das, Maya Singh, Sneha Reddy, Maya Patel, Riya Sharma) was that the pace of the course was too fast.
*   **Outdated Content:** Several reviewers (Maya Singh, Kunal Reddy, Rohit Sharma) felt that some topics were outdated and irrelevant.
*   **Outdated Lab Instruments:*

In [47]:
cluster_labels = {}
for i, summary in cluster_summaries.items():
    prompt = f"Based on this summary, provide one short unique category label to each cluster\n{summary}"
    response = llm.invoke(prompt)
    cluster_labels[i] = response
    print(f" Cluster {i} Label: {response}")

 Cluster 0 Label: Here are the category labels:

*   **Positive Feedback:**
    *   Clear Explanations: **Instruction Clarity**
    *   Fair Exams: **Assessment Fairness**
    *   Real-World Case Studies: **Practical Application**
    *   Well Structured Syllabus: **Course Organization**

*   **Negative Feedback/Areas for Improvement:**
    *   Pace Too Fast: **Pace & Delivery**
    *   Outdated Content: **Content Relevance**
    *   Outdated Lab Instruments: **Resource Quality**
    *   Unreliable Internet: **Technical Issues**
    *   Noise Issues: **Environment**
    *   Limited Help Outside Class: **Support Availability**
 Cluster 1 Label: Here are the unique category labels:

*   **Positive Aspects:** **Learning Experience**
*   **Areas for Improvement:** **Support & Structure**
 Cluster 2 Label: Here are the unique category labels:

*   **Positive Aspects:** **Effective Instruction**
*   **Negative Aspects & Areas for Improvement:** **Areas for Improvement**
 Cluster 3 Label: Her

In [48]:
#optional code 
#i want docx of summary:
from docx import Document

# Create a new Word document
doc = Document()
doc.add_heading("Clustered Student Feedback Summary", 0)

# Loop through clusters and add content
for i in range(len(cluster_summaries)):
    label = cluster_labels.get(i, f"Cluster {i}")
    summary = cluster_summaries.get(i, "No summary available.")

    doc.add_heading(f"Cluster {i + 1}: {label}", level=1)
    doc.add_paragraph(summary)
    doc.add_paragraph("\n")  # Add space between clusters

# Save the document
output_path = "clustered_feedback_summary.docx"
doc.save(output_path)

print(f"docx file saved to: {output_path}")

docx file saved to: clustered_feedback_summary.docx
